# Google Setup

In [27]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

google_project_name = "msd8654-498-dev"
google_dataset_name = "usgs"

Authenticated


# Get data from usgs.gov

In [34]:
from google.cloud import bigquery

bqclient = bigquery.Client()
#SELECT state_postal_abbreviation FROM `bigquery-public-data.census_utility.fips_codes_states` 
# Download a table.
table = bigquery.TableReference.from_string(
    "bigquery-public-data.census_utility.fips_codes_states"
)
rows = bqclient.list_rows(
    table
)
df = rows.to_dataframe()

# continental states and HI/AK al under 60. US Virgin islands, PR, Guam filtered
df = df[df.state_fips_code < '60']
fips_codes_states = df["state_postal_abbreviation"]
print(fips_codes_states.head())

0    AL
1    AK
2    AZ
3    AR
4    CA
Name: state_postal_abbreviation, dtype: object


In [29]:
from six import b
import requests
import json
import pandas as pd
from google.cloud import bigquery
import urllib.request
import os

base_url = "https://nwis.waterdata.usgs.gov/nwis/gwlevels?state_cd={state_cd}"
base_url = base_url + "&group_key=NONE&format=sitefile_output&sitefile_output_format=rdb"
base_url = base_url + "&date_format=YYYY-MM-DD&rdb_compression=file&list_of_search_criteria=state_cd"
column_list = ['agency_cd', 'site_no', 'station_nm', 'site_tp_cd', 'lat_va', 'long_va',
    'dec_lat_va', 'dec_long_va', 'coord_meth_cd', 'coord_acy_cd', 'coord_datum_cd', 
    'dec_coord_datum_cd', 'district_cd', 'state_cd', 'county_cd', 'country_cd', 
    'land_net_ds', 'map_nm', 'map_scale_fc', 'alt_va', 'alt_meth_cd', 'alt_acy_va',
    'alt_datum_cd', 'huc_cd', 'basin_cd', 'topo_cd', 'data_types_cd', 'instruments_cd',
    'construction_dt', 'inventory_dt', 'drain_area_va', 'contrib_drain_area_va', 'tz_cd', 
    'local_time_fg', 'reliability_cd', 'gw_file_cd', 'nat_aqfr_cd', 'aqfr_cd', 'aqfr_type_cd',
    'well_depth_va', 'hole_depth_va', 'depth_src_cd', 'project_no', 'rt_bol', 'peak_begin_date',
    'peak_end_date', 'peak_count_nu', 'qw_begin_date', 'qw_end_date', 'qw_count_nu',
    'gw_begin_date', 'gw_end_date', 'gw_count_nu', 'sv_begin_date', 'sv_end_date', 'sv_count_nu']

for c in column_list:
  base_url = base_url +"&" + c
# print(base_url)

for state_cd in fips_codes_states:
  f = open("groundwater_sites_" + state_cd + ".tmp", 'w')
  f2 = open("groundwater_sites_" + state_cd + ".tsv", 'w')
  url = base_url.replace("{state_cd}",state_cd.lower())
  payload={}
  headers = {}
  response = requests.request("GET", url, headers=headers, data=payload)
  print(state_cd,"Response code:", response.status_code) 
  if (response.status_code>229):
    print("ERROR")
    break;
  else:
    f.writelines(response.text)
    f.close()

  for line in open("groundwater_sites_" + state_cd + ".tmp", 'r'):
    if (line.startswith("#") or line.startswith("5s")):
      #print(line)
      None
    else:
      f2.writelines(line)
  f2.close()

AL Response code: 200
AK Response code: 200
AZ Response code: 200
AR Response code: 200
CA Response code: 200
CO Response code: 200
CT Response code: 200
DE Response code: 200
DC Response code: 200
FL Response code: 200
GA Response code: 200
HI Response code: 200
ID Response code: 200
IL Response code: 200
IN Response code: 200
IA Response code: 200
KS Response code: 200
KY Response code: 200
LA Response code: 200
ME Response code: 200
MD Response code: 200
MA Response code: 200
MI Response code: 200
MN Response code: 200
MS Response code: 200
MO Response code: 200
MT Response code: 200
NE Response code: 200
NV Response code: 200
NH Response code: 200
NJ Response code: 200
NM Response code: 200
NY Response code: 200
NC Response code: 200
ND Response code: 200
OH Response code: 200
OK Response code: 200
OR Response code: 200
PA Response code: 200
RI Response code: 200
SC Response code: 200
SD Response code: 200
TN Response code: 200
TX Response code: 200
UT Response code: 200
VT Respons

# Upload file to GCS Bucket

In [30]:
from google.cloud import storage
from google.cloud import bigquery
import urllib.request
import os

# note: google_project_name  & google_dataset_name at top (global)

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the google storage bucket."""

    storage_client = storage.Client(project=google_project_name)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to Storage Bucket with Bob name  {} successfully . {}".format(
            source_file_name, destination_blob_name, datetime.datetime.now()
        )
    )

# if running locally will need app credentials 
# terminal
# GOOGLE_APPLICATION_CREDENTIALS=<path>/<filename>.json
# powershell:
# $env:GOOGLE_APPLICATION_CREDENTIALS="<path>/<filename>.json"

# List the tsv files and load to BQ
from os import listdir
from os.path import isfile, join
files = [f for f in os.listdir() if (isfile(f))]
bucket_name = google_project_name + "-" + google_dataset_name
for f in files:
 try: 
    upload_blob(bucket_name, f , f)
 except BaseException as err:
    print(f,type(err))
    print(err)

File groundwater_sites_WV.tsv uploaded to Storage Bucket with Bob name  groundwater_sites_WV.tsv successfully . 2022-04-24 21:48:34.241240
File groundwater_sites_AS.tmp uploaded to Storage Bucket with Bob name  groundwater_sites_AS.tmp successfully . 2022-04-24 21:48:34.565988
File groundwater_sites_WY.tmp uploaded to Storage Bucket with Bob name  groundwater_sites_WY.tmp successfully . 2022-04-24 21:48:34.896009
File groundwater_sites_AR.tmp uploaded to Storage Bucket with Bob name  groundwater_sites_AR.tmp successfully . 2022-04-24 21:48:35.240010
File groundwater_sites_NC.tsv uploaded to Storage Bucket with Bob name  groundwater_sites_NC.tsv successfully . 2022-04-24 21:48:35.564103
File groundwater_sites_AZ.tsv uploaded to Storage Bucket with Bob name  groundwater_sites_AZ.tsv successfully . 2022-04-24 21:48:35.960900
File groundwater_sites_GU.tsv uploaded to Storage Bucket with Bob name  groundwater_sites_GU.tsv successfully . 2022-04-24 21:48:36.267171
File groundwater_sites_IA.t

# Upload files to BigQuery Table

In [31]:
# https://cloud.google.com/bigquery/docs/batch-loading-data#bq
import os, sys, datetime
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project=google_project_name)
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV, 
    fieldDelimiter="\t",
    skip_leading_rows=1, 
    autodetect=True
)

# List the tsv files and load to BQ
from os import listdir
from os.path import isfile, join
files = [f for f in os.listdir() if (isfile(f) and f.endswith(".tsv"))]

for f in files:
  table_name = f.split(".")[0]
  table_id = google_project_name + "." + google_dataset_name + "." + table_name
  with open(f, "rb") as source_file:
    try:
      job = client.load_table_from_file(source_file, table_id, job_config=job_config)

      job.result()  # Waits for the job to complete.

      table = client.get_table(table_id)  # Make an API request.
      print(
          "Loaded {} rows and {} columns to {} at {}".format(
              table.num_rows, len(table.schema), table_id, datetime.datetime.now())
          )
    except BaseException as err:
      print(f, table_name, type(err))
      print(err)


Loaded 21856 rows and 15 columns to msd8654-498-dev.usgs.groundwater_sites_WV at 2022-04-24 21:49:22.715989
Loaded 27054 rows and 15 columns to msd8654-498-dev.usgs.groundwater_sites_NC at 2022-04-24 21:49:27.859077
Loaded 110022 rows and 15 columns to msd8654-498-dev.usgs.groundwater_sites_AZ at 2022-04-24 21:49:41.698048
Loaded 282 rows and 15 columns to msd8654-498-dev.usgs.groundwater_sites_GU at 2022-04-24 21:49:45.378931
Loaded 23649 rows and 15 columns to msd8654-498-dev.usgs.groundwater_sites_IA at 2022-04-24 21:49:50.323709
Loaded 14787 rows and 15 columns to msd8654-498-dev.usgs.groundwater_sites_AL at 2022-04-24 21:49:55.480424
Loaded 37146 rows and 15 columns to msd8654-498-dev.usgs.groundwater_sites_NV at 2022-04-24 21:49:59.132264
Loaded 227403 rows and 15 columns to msd8654-498-dev.usgs.groundwater_sites_WA at 2022-04-24 21:50:10.658726
Loaded 52578 rows and 15 columns to msd8654-498-dev.usgs.groundwater_sites_MA at 2022-04-24 21:50:16.524934
Loaded 8241 rows and 15 colu